Setup

In [8]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc

plt.ion()   # interactive mode

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Model with spatial transformation

In [58]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

        # Spatial transformer localization-network
        self.localization = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )

        # Regressor for the 3 * 2 affine matrix
        self.fc_loc = nn.Sequential(
            nn.Linear(10 * 3 * 3, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )

        # Initialize the weights/bias with identity transformation
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    # Spatial transformer network forward function
    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 10 * 3 * 3)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)

        return x

    def forward(self, x):
        # transform the input
        x = self.stn(x)

        # Perform the usual forward pass
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


model = Net().to(device)

Declare other functions

In [37]:
def train(epoch, train_loader):
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    model.train()
    total_loss = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
#
# A simple test procedure to measure STN the performances on MNIST.
#

def test(test_loader):
    with torch.no_grad():
        model.eval()
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # sum up batch loss
            test_loss += F.nll_loss(output, target, size_average=False).item()
            # get the index of the max log-probability
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'
              .format(test_loss, correct, len(test_loader.dataset),
                      100. * correct / len(test_loader.dataset)))

######################################################################
# Visualizing the STN results
# ---------------------------
#
# Now, we will inspect the results of our learned visual attention
# mechanism.
#
# We define a small helper function in order to visualize the
# transformations while training.


def convert_image_np(inp):
    """Convert a Tensor to numpy image."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    return inp

# We want to visualize the output of the spatial transformers layer
# after the training, we visualize a batch of input images and
# the corresponding transformed batch using STN.


def visualize_stn(test_loader):
    with torch.no_grad():
        # Get a batch of training data
        data = next(iter(test_loader))[0].to(device)

        input_tensor = data.cpu()
        transformed_input_tensor = model.stn(data).cpu()

        in_grid = convert_image_np(
            torchvision.utils.make_grid(input_tensor))

        out_grid = convert_image_np(
            torchvision.utils.make_grid(transformed_input_tensor))

        # scipy.misc.imsave('in_grid.png', in_grid)
        # scipy.misc.imsave('out_grid.png', out_grid)
        torchvision.utils.save_image(input_tensor, "in_grid_img.png", nrow=8, padding=2, normalize=False, range=None, scale_each=False,
                                     pad_value=0)

        torchvision.utils.save_image(transformed_input_tensor, "out_grid_img.png", nrow=8, padding=2, normalize=False, range=None,
                                     scale_each=False,
                                     pad_value=0)

Importing MNIST and distort it

In [32]:
# Training dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='./datasets', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.RandomAffine(45),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])), batch_size=64, shuffle=True, num_workers=4)
# Test dataset
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='./datasets', train=False, transform=transforms.Compose([
        transforms.RandomAffine(45),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])), batch_size=64, shuffle=True, num_workers=4)

Importing the GTSRB and distort it

In [33]:
from torch.utils.data import Dataset
import csv
from PIL import Image

path_train = './data/train/GTSRB/Final_Training/Images'
path_test = "./data/test/GTSRB/Final_Test/Images"


class TrafficSignals(Dataset):
    """
    Dataset wrapping images and target labels for Kaggle - Planet Amazon from Space competition.

    Arguments:
        A CSV file path
        Path to image folder
        Extension of images
        PIL transforms
    """

    def __init__(self, path_train, path_test, train=False, transform=None):

        self.path_train = path_train
        self.path_test = path_test
        self.transform = transform
        self.train = train

        if train:
            [self.X_train, self.y_train] = self.readTrafficSigns(path_train, train=True)

        else:
            [self.X_train, self.y_train] = self.readTrafficSigns(path_test, train=False)

        # print(self.y_train)
        # print(self.X_train)

    def __getitem__(self, index):
        img = Image.open(self.X_train[index]).resize((28,28), resample=0)
        if self.transform is not None:
             img = self.transform(img)
        label = int(self.y_train[index])

        return img, label

    def __len__(self):
        return len(self.X_train)

    def readTrafficSigns(self, rootpath, train=False):
        '''Reads traffic sign data for German Traffic Sign Recognition Benchmark.

        Arguments: path to the traffic sign data, for example './GTSRB/Training'
        Returns:   list of images, list of corresponding labels'''
        images = []  # images
        labels = []  # corresponding labels

        if train:

        # loop over all 42 classes
            for c in range(0, 43):
                # prefix = rootpath + '/'  # subdirectory for class
                prefix = rootpath + '/' + format(c, '05d') + '/' # subdirectory for class
                gtFile = open(prefix + 'GT-'+ format(c, '05d') + '.csv') # annotations file
                gtReader = csv.reader(gtFile, delimiter=';')  # csv parser for annotations file
                next(gtReader)  # skip header
                # loop over all images in current annotations file
                for row in gtReader:
                    images.append(prefix + row[0])  # the 1th column is the filename
                    # images.append(plt.imread(prefix + row[0]))  # the 1th column is the filename
                    labels.append(row[7])  # the 8th column is the label
                gtFile.close()

        else:
            prefix = rootpath + '/' # subdirectory for class
            gtFile = open(prefix + 'GT-final_test.csv') # annotations file
            gtReader = csv.reader(gtFile, delimiter=';')  # csv parser for annotations file
            next(gtReader)  # skip header
            # loop over all images in current annotations file
            for row in gtReader:
                images.append(prefix + row[0])  # the 1th column is the filename
                # images.append(plt.imread(prefix + row[0]))  # the 1th column is the filename
                labels.append(row[7])  # the 8th column is the label
            gtFile.close()
        return images, labels

    
#train dataset loader
dset_train = TrafficSignals(path_train, path_test, train=True, transform=transforms.Compose([
                       transforms.Grayscale(),
                       transforms.RandomAffine(45),
                       transforms.ToTensor(),
                   ]))
print(dset_train.__getitem__(1))
dset_train = torch.utils.data.DataLoader(dset_train,
                          batch_size=64,
                          shuffle=True,
                          num_workers=4
                         # pin_memory=True # CUDA only
                         )

#test dataset loader
dset_test = TrafficSignals(path_train, path_test, transform=transforms.Compose([
                       transforms.Grayscale(),
                    transforms.RandomAffine(45),
                       transforms.ToTensor(),
                   ]))
dset_test = torch.utils.data.DataLoader(dset_test,
                          batch_size=64,
                          shuffle=True,
                          num_workers=4
                         # pin_memory=True # CUDA only
                         )

(tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.8863,  0.8078,  0.8078,  0.5020,  0.4667,
           0.3882,  0.4157,  0.4157,  0.3882,  0.2784,  0.2510,  0.2863,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.9216,  0.9529,  0.9412,  0.9412,  0.5098,  0.5098,
           0.3608,  0.3373,  0.4824,  0.3216,  0.3059,  0.2745,  0.2824,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.6706,
           0.7961,  0.8941,  0.9725,  0.9725,  0.7608,  0.6667,  0.3725,
           0.4745,  0.4745,  0.3333,  0.2980,  0.2745,  0.2863,  0.2784,
           0.2706,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.7490,  0.6706,
           0.6471,  0.5922,  0.7882,  0.8000,  

Train and test for the MNIST dataset

In [42]:
for epoch in range(1, 20 + 1):
    train(epoch, train_loader)
    test(test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.190533
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.383691

Test set: Average loss: 0.1011, Accuracy: 9685/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.041331
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.117734

Test set: Average loss: 0.0738, Accuracy: 9773/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.161919
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.241965

Test set: Average loss: 0.0638, Accuracy: 9808/10000 (98%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.154912
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.119330

Test set: Average loss: 0.0726, Accuracy: 9768/10000 (98%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.187385
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.208705

Test set: Average loss: 0.0677, Accuracy: 9786/10000 (98%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.189368
Train Epoch: 6 [32000/60000 (53%)]	Loss: 0.144360

Test set: Average loss: 0.0620, Accuracy: 9805/10000 (98%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.242441
Train Epoch: 7 [320

Train and test for GTSRB dataset

In [50]:
for epoch in range(1, 20 + 1):
    train(epoch, dset_train)
    test(dset_test)

Train Epoch: 1 [0/39209 (0%)]	Loss: 3.754161
Train Epoch: 1 [32000/39209 (82%)]	Loss: 3.661552

Test set: Average loss: 3.5177, Accuracy: 720/12630 (6%)

Train Epoch: 2 [0/39209 (0%)]	Loss: 3.596776
Train Epoch: 2 [32000/39209 (82%)]	Loss: 3.563682

Test set: Average loss: 3.4819, Accuracy: 720/12630 (6%)

Train Epoch: 3 [0/39209 (0%)]	Loss: 3.594419
Train Epoch: 3 [32000/39209 (82%)]	Loss: 3.633026

Test set: Average loss: 3.4692, Accuracy: 720/12630 (6%)

Train Epoch: 4 [0/39209 (0%)]	Loss: 3.612273
Train Epoch: 4 [32000/39209 (82%)]	Loss: 3.602080

Test set: Average loss: 3.4597, Accuracy: 775/12630 (6%)

Train Epoch: 5 [0/39209 (0%)]	Loss: 3.435731
Train Epoch: 5 [32000/39209 (82%)]	Loss: 3.431840

Test set: Average loss: 3.4308, Accuracy: 1067/12630 (8%)

Train Epoch: 6 [0/39209 (0%)]	Loss: 3.606174
Train Epoch: 6 [32000/39209 (82%)]	Loss: 3.430386

Test set: Average loss: 3.4001, Accuracy: 1372/12630 (11%)

Train Epoch: 7 [0/39209 (0%)]	Loss: 3.490295
Train Epoch: 7 [32000/39209 

Model without spatial transformation

In [66]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        # Perform the usual forward pass
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


model = Net().to(device)

Train and test for the MNIST dataset

In [61]:
for epoch in range(1, 20 + 1):
    train(epoch, train_loader)
    test(test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.326681
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.637515

Test set: Average loss: 0.7005, Accuracy: 8098/10000 (81%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.154671
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.865778

Test set: Average loss: 0.4183, Accuracy: 8816/10000 (88%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.724624
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.709928

Test set: Average loss: 0.2993, Accuracy: 9183/10000 (92%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.518698
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.523415

Test set: Average loss: 0.2556, Accuracy: 9264/10000 (93%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.396971
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.447880

Test set: Average loss: 0.2205, Accuracy: 9350/10000 (94%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.550620
Train Epoch: 6 [32000/60000 (53%)]	Loss: 0.453717

Test set: Average loss: 0.2054, Accuracy: 9391/10000 (94%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.569704
Train Epoch: 7 [320

Train and test for GTSRB dataset

In [65]:
for epoch in range(1, 20 + 1):
    train(epoch, dset_train)
    test(dset_test)

Train Epoch: 1 [0/39209 (0%)]	Loss: 3.778571
Train Epoch: 1 [32000/39209 (82%)]	Loss: 3.589249

Test set: Average loss: 3.5688, Accuracy: 720/12630 (6%)

Train Epoch: 2 [0/39209 (0%)]	Loss: 3.641155
Train Epoch: 2 [32000/39209 (82%)]	Loss: 3.576416

Test set: Average loss: 3.5053, Accuracy: 544/12630 (4%)

Train Epoch: 3 [0/39209 (0%)]	Loss: 3.586509
Train Epoch: 3 [32000/39209 (82%)]	Loss: 3.570819

Test set: Average loss: 3.4806, Accuracy: 734/12630 (6%)

Train Epoch: 4 [0/39209 (0%)]	Loss: 3.482228
Train Epoch: 4 [32000/39209 (82%)]	Loss: 3.511797

Test set: Average loss: 3.4621, Accuracy: 893/12630 (7%)

Train Epoch: 5 [0/39209 (0%)]	Loss: 3.595092
Train Epoch: 5 [32000/39209 (82%)]	Loss: 3.590954

Test set: Average loss: 3.4461, Accuracy: 1142/12630 (9%)

Train Epoch: 6 [0/39209 (0%)]	Loss: 3.557591
Train Epoch: 6 [32000/39209 (82%)]	Loss: 3.406265

Test set: Average loss: 3.4193, Accuracy: 1232/12630 (10%)

Train Epoch: 7 [0/39209 (0%)]	Loss: 3.388220
Train Epoch: 7 [32000/39209 